In [1]:
#!/usr/bin/env python3
"""
Download a central cube from JHTDB (isotropic1024_fine) with gradient operator.
Choose spatialMethod automatically: prefer m2q8, fallback to fd8noint, fd6noint.
Saves HDF5 with velocity and gradient (if available) and computes invariants (vort, Q, lambda2).
"""

import numpy as np
import h5py
import sys
import os

# ---------- USER PARAMETERS ----------
DATASET = "isotropic1024_fine"   # change if your dataset has another exact name
NX = 128                         # cube size (128 recommended)
T = 5.0                          # snapshot time (in dataset units)
SPATIAL_OPERATOR = "gradient"    # request gradients from server if possible
PREFERRED_METHODS = ["m2q8", "fd8noint", "fd6noint", "m1q4", "fd4noint"]
OUTDIR = "src/analysis/jhtdb"
OUTNAME = f"{DATASET}_center_{NX}c_t{T:.2f}.h5"
os.makedirs(OUTDIR, exist_ok=True)
OUTPATH = os.path.join(OUTDIR, OUTNAME)
# -------------------------------------

# Helper: compute central index bounds for N=1024
N_FULL = 1024
i0 = N_FULL//2 - NX//2
i1 = i0 + NX - 1

# Physical coords (if client needs coords)
L = 2.0 * np.pi
dx = L / N_FULL
x0 = i0 * dx
x1 = (i1 + 1) * dx
# same for y,z (centered cube)
y0, y1 = x0, x1
z0, z1 = x0, x1

print("Preparing request:")
print(f" Dataset: {DATASET}")
print(f" Cube indices i0..i1: {i0}..{i1} (NX={NX})")
print(f" Physical coords x: {x0:.6f} .. {x1:.6f}")
print(f" Time t = {T}")

# ---------- Attempt to use pyJHTDB / JHTDB client ----------
# If you have the official python client installed, uncomment and use it.
# Installation (if available): pip install py_jhtdb   (package name may vary)
#
# from JHTDB import JHTDBClient   # <--- adjust this import to your installed client
#
# client = JHTDBClient()
# available_methods = client.list_spatial_methods(dataset=DATASET, op=SPATIAL_OPERATOR)
# print("Available spatial methods:", available_methods)
#
# # choose method
# chosen = None
# for m in PREFERRED_METHODS:
#     if m in available_methods:
#         chosen = m
#         break
# if chosen is None:
#     raise RuntimeError("No preferred spatial method available. Check available_methods.")
# print("Chosen spatial method:", chosen)
#
# # Request block (client API will vary; adapt below)
# # Example pseudo-call (replace with actual client call):
# # data = client.getData(dataset=DATASET, field='velocity', 
# #                      spatialOperator=SPATIAL_OPERATOR, spatialMethod=chosen,
# #                      temporalMethod='none', x=[x0,x1], y=[y0,y1], z=[z0,z1], t=T)
#
# # Expect 'data' to contain arrays for velocity and gradient if requested.
# # Then save to HDF5 and compute invariants as below.

# ---------- Fallback: Prompt user to perform request via JHTDB web UI ----------
print("\n*** NOTE ***")
print("This script is a template. If you do not have the pyJHTDB client installed,")
print("please perform the matching request in the JHTDB web interface using these parameters:")
print(f" - Dataset: {DATASET}")
print(f" - Field: velocity")
print(f" - Spatial operator: {SPATIAL_OPERATOR}")
print(" - Choose spatial method in this order of preference:", ", ".join(PREFERRED_METHODS))
print(f" - Temporal method: none (snapshot)")
print(f" - Spatial limits: use indices {i0}..{i1} in x,y,z (central cube) OR coords {x0:.6f}..{x1:.6f}")
print(f" - Time t = {T}")
print("\nAfter you obtain the raw arrays (velocity and optionally gradient),")
print("place them in an HDF5 file with datasets: '/velocity' and '/gradA' and then re-run the")
print("invariant computation section below (uncommented).")
print("This template will then compute vorticity, Q, lambda2 and save outputs.")

# ---------- If you already have HDF5 with velocity (and optionally gradA), compute invariants ----------
# For convenience, the script below will look for an HDF5 at OUTPATH. If found, it will process it.
if not os.path.exists(OUTPATH):
    print(f"\nHDF5 file {OUTPATH} not found. Please run the JHTDB request first (web UI or client) and")
    print("save the returned arrays to the above path with datasets 'velocity' and optionally 'gradA'.")
    print("Exiting template.")
    sys.exit(0)

print("Found HDF5 file, loading and computing invariants...")

with h5py.File(OUTPATH, 'r+') as f:
    vel = f['velocity'][:]   # shape (NX,NX,NX,3)
    has_grad = 'gradA' in f
    if has_grad:
        gradA = f['gradA'][:]  # shape (NX,NX,NX,3,3)
    else:
        gradA = None

# If gradA not present, compute gradients numerically (central FD) -- expensive
if gradA is None:
    print("gradA not found: computing numerical gradients via spectral/FD (may be slow).")
    # naive finite differences (periodic) as fallback:
    u = vel[...,0]; v = vel[...,1]; w = vel[...,2]
    # compute derivatives along each axis using numpy.gradient (not as accurate as server)
    du_dx, du_dy, du_dz = np.gradient(u, dx, dx, dx, edge_order=2)
    dv_dx, dv_dy, dv_dz = np.gradient(v, dx, dx, dx, edge_order=2)
    dw_dx, dw_dy, dw_dz = np.gradient(w, dx, dx, dx, edge_order=2)
    gradA = np.empty((NX,NX,NX,3,3), dtype=u.dtype)
    gradA[...,0,0] = du_dx; gradA[...,0,1] = du_dy; gradA[...,0,2] = du_dz
    gradA[...,1,0] = dv_dx; gradA[...,1,1] = dv_dy; gradA[...,1,2] = dv_dz
    gradA[...,2,0] = dw_dx; gradA[...,2,1] = dw_dy; gradA[...,2,2] = dw_dz

# Compute invariants
# vorticity magnitude ||omega||
Omega = 0.5 * (gradA - np.swapaxes(gradA, -1, -2))  # shape (...,3,3)
S = 0.5 * (gradA + np.swapaxes(gradA, -1, -2))

# vorticity vector (omega_i = eps_{ijk} A_{kj})
omega_x = gradA[...,2,1] - gradA[...,1,2]
omega_y = gradA[...,0,2] - gradA[...,2,0]
omega_z = gradA[...,1,0] - gradA[...,0,1]
vort_mag = np.sqrt(omega_x**2 + omega_y**2 + omega_z**2)

# traces
tr_O2 = np.sum(Omega**2, axis=(-2,-1))
tr_S2 = np.sum(S**2, axis=(-2,-1))
Q = 0.5 * (tr_O2 - tr_S2)

# lambda2: compute eigenvalues of S^2 + Omega^2 (small 3x3 matrices)
# Build M = S^2 + Omega^2 at each point
M = np.einsum('...ij,...jk->...ik', S, S) + np.einsum('...ij,...jk->...ik', Omega, Omega)
# reshape to (-1,3,3) for eig calc
Mflat = M.reshape(-1,3,3)
eigvals = np.linalg.eigvalsh(Mflat)
# second largest eigenvalue (lambda2)
lambda2_flat = np.sort(eigvals, axis=1)[:,1]
lambda2 = lambda2_flat.reshape(NX,NX,NX)

# Save invariants back to HDF5
with h5py.File(OUTPATH, 'a') as f:
    if 'vorticity' not in f:
        f.create_dataset('vorticity', data=vort_mag, compression='gzip')
    if 'Q' not in f:
        f.create_dataset('Q', data=Q, compression='gzip')
    if 'lambda2' not in f:
        f.create_dataset('lambda2', data=lambda2, compression='gzip')

print("Invariants computed and saved to HDF5. Next: run clustering / kmeans on these fields.")


Preparing request:
 Dataset: isotropic1024_fine
 Cube indices i0..i1: 448..575 (NX=128)
 Physical coords x: 2.748894 .. 3.534292
 Time t = 5.0

*** NOTE ***
This script is a template. If you do not have the pyJHTDB client installed,
please perform the matching request in the JHTDB web interface using these parameters:
 - Dataset: isotropic1024_fine
 - Field: velocity
 - Spatial operator: gradient
 - Choose spatial method in this order of preference: m2q8, fd8noint, fd6noint, m1q4, fd4noint
 - Temporal method: none (snapshot)
 - Spatial limits: use indices 448..575 in x,y,z (central cube) OR coords 2.748894..3.534292
 - Time t = 5.0

After you obtain the raw arrays (velocity and optionally gradient),
place them in an HDF5 file with datasets: '/velocity' and '/gradA' and then re-run the
invariant computation section below (uncommented).
This template will then compute vorticity, Q, lambda2 and save outputs.

HDF5 file src/analysis/jhtdb/isotropic1024_fine_center_128c_t5.00.h5 not found. 

SystemExit: 0

/home/mesh/Multiflow-Turbulence/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
